<a href="https://colab.research.google.com/github/cl49/EPU_ML_Angers_2023/blob/main/Jour%203/2_CT_Image_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CT Image Comparison Using PSNR and SSIM

This notebook provides a Python script to compare two CT images - one real and one synthetic - using the PSNR (Peak Signal-to-Noise Ratio) and SSIM (Structural Similarity Index Measure) metrics. The images are expected to be in the NIfTI format (.nii).

First, we need to import the necessary libraries.

The `read_nifti_file` function reads a NIfTI file and converts it into a 2D numpy array. NIfTI files can be 3D or 4D; this function takes the first slice for simplicity.

The `calculate_psnr` function computes the Peak Signal-to-Noise Ratio (PSNR) between two images, which is a measure of the reconstruction quality compared to the original image.

The `calculate_ssim` function calculates the Structural Similarity Index Measure (SSIM) between two images, a metric used to measure the similarity between two images.

Below is the example usage of the functions. You need to replace `'path/to/real_ct.nii'` and `'path/to/synthetic_ct.nii'` with the actual paths to your NIfTI files. Make sure to have `nibabel`, `opencv-python`, and `scikit-image` installed in your Python environment.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
import nibabel as nib

In [10]:
def read_nifti_file(filepath):
    nifti_data = nib.load(filepath)
    image = nifti_data.get_fdata()
    if len(image.shape) == 4:
        image = image[:, :, 0, 0]
    return image

In [11]:
# Example usage
real_image = read_nifti_file('/content/drive/MyDrive/Formation_ICO/Jour3/image.nii.gz')
synthetic_image = read_nifti_file('/content/drive/MyDrive/Formation_ICO/Jour3/sCT_GA1.nii.gz')

In [12]:
real_image.shape

(256, 256, 128)

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def visualize_interactive_ct(image):
    """
    Visualizes a 3D CT image with an interactive slider to navigate through slices.

    Parameters:
    image (numpy.ndarray): The 3D image to visualize.
    """

    # Define a function to update the plot with the selected slice
    def update_slice(slice_index):
        rotated_slice = np.rot90(image[:, :, slice_index],k=3)
        plt.imshow(rotated_slice, cmap='gray')
        plt.title(f'Slice {slice_index}')
        plt.axis('off')
        plt.show()

    # Create an interactive slider
    slice_slider = widgets.IntSlider(
        value=0,
        min=0,
        max=image.shape[2] - 1,
        step=1,
        description='Slice:',
        continuous_update=False
    )

    # Create an interactive widget using the slider
    widgets.interact(update_slice, slice_index=slice_slider)


In [33]:
visualize_interactive_ct(real_image)
visualize_interactive_ct(synthetic_image)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Slice:', max=127), Output()), _…

interactive(children=(IntSlider(value=0, continuous_update=False, description='Slice:', max=127), Output()), _…

In [16]:
def calculate_psnr(image1, image2):
    return cv2.PSNR(image1, image2)

In [17]:
def calculate_ssim(image1, image2):
    return ssim(image1, image2, data_range=image2.max() - image2.min())

In [18]:
real_image = cv2.normalize(real_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
synthetic_image = cv2.normalize(synthetic_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

psnr_value = calculate_psnr(real_image, synthetic_image)
ssim_value = calculate_ssim(real_image, synthetic_image)

print(f"PSNR: {psnr_value}")
print(f"SSIM: {ssim_value}")

PSNR: 19.480288058432144
SSIM: 0.8507500348461902


The `calculate_mse` function computes the Mean Squared Error (MSE) between two images. MSE is a measure of the average squared difference between corresponding pixels in two images. A lower MSE indicates higher similarity.

In [19]:
def calculate_mse(image1, image2):
    err = np.sum((image1.astype("float") - image2.astype("float")) ** 2)
    err /= float(image1.shape[0] * image1.shape[1])
    return err

mse_value = calculate_mse(real_image, synthetic_image)

print(f"mse: {mse_value}")


mse: 93812.67606776125


The `calculate_rmse` function calculates the Root Mean Squared Error (RMSE) between two images. RMSE is the square root of the MSE and provides a measure of the magnitude of error between the images.

In [20]:
def calculate_rmse(image1, image2):
    mse = calculate_mse(image1, image2)
    return np.sqrt(mse)

rmse_value = calculate_rmse(real_image, synthetic_image)

print(f"rmse: {rmse_value}")

rmse: 306.2885503373596


The `calculate_mae` function calculates the Mean Absolute Error (MAE) between two images. MAE measures the average magnitude of errors between two images, without considering their direction.

In [21]:
def calculate_mae(image1, image2):
    err = np.sum(abs(image1.astype("float") - image2.astype("float")))
    err /= float(image1.shape[0] * image1.shape[1])
    return err

mae_value = calculate_mae(real_image, synthetic_image)

print(f"mae: {mae_value}")

mae: 2734.410607811231


The `calculate_pearson` function computes the Pearson Correlation Coefficient between two images. This coefficient measures the linear correlation between the images, with 1 indicating total positive linear correlation and -1 indicating total negative linear correlation.

In [22]:
from scipy.stats import pearsonr
def calculate_pearson(image1, image2):
    corr, _ = pearsonr(image1.flatten(), image2.flatten())
    return corr

pearson_value = calculate_pearson(real_image, synthetic_image)

print(f"pearson: {pearson_value}")

pearson: 0.9681007309561759


The `calculate_mutual_information` function calculates Mutual Information for two images. Mutual Information measures the amount of information that one image contains about the other, making it useful for assessing the similarity of images.

In [23]:
from sklearn.metrics import mutual_info_score
def calculate_mutual_information(image1, image2):
    return mutual_info_score(None, None, contingency=np.histogram2d(image1.flatten(), image2.flatten(), bins=20)[0])

mutual_value = calculate_mutual_information(real_image, synthetic_image)

print(f"mutual information: {mutual_value}")


mutual information: 1.1984290834602813
